# Resistance/Support AND Candles Patterns
# MARTINGALE DEMO

In [13]:
import pandas as pd
df = pd.read_csv("BTCUSD_Candlestick_1_D_BID_01.10.2019-24.09.2022.csv")

#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,Local time,Open,High,Low,Close,Volume
1017,20.09.2022 14:00:00.000 GMT-0700,18936,19845,18694,18891,0.8975
1018,21.09.2022 14:00:00.000 GMT-0700,18891,19449,18140,19212,1.1559
1019,22.09.2022 14:00:00.000 GMT-0700,19212,19465,18501,18811,0.8004
1020,23.09.2022 14:00:00.000 GMT-0700,18811,19335,18743,19086,0.0584
1021,24.09.2022 14:00:00.000 GMT-0700,19086,19130,18777,18851,0.1035


# Support and Resistance FUNCTIONS

In [22]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [27]:
length = len(df)
high = list(df['High'])
low = list(df['Low'])
close = list(df['Close'])
open = list(df['Open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    c4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    c4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [28]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [29]:
df['signal']=signal

In [30]:
df[df['signal']==2].count()

Local time    13
Open          13
High          13
Low           13
Close         13
Volume        13
signal        13
dtype: int64

In [31]:
df.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']
#df=df.iloc[100:200]
df

,Local time,Open,High,Low,Close,Volume,signal
0,30.09.2019 14:00:00.000 GMT-0700,8181,8472,8137,8318,601.2124,0
1,01.10.2019 14:00:00.000 GMT-0700,8318,8359,8120,8176,352.9879,0
2,02.10.2019 14:00:00.000 GMT-0700,8176,8359,8030,8091,264.0550,0
3,03.10.2019 14:00:00.000 GMT-0700,8088,8239,7972,8109,287.4896,0
4,06.10.2019 14:00:00.000 GMT-0700,7850,8254,7714,8159,536.0352,0
...,...,...,...,...,...,...,...
1017,20.09.2022 14:00:00.000 GMT-0700,18936,19845,18694,18891,0.8975,0
1018,21.09.2022 14:00:00.000 GMT-0700,18891,19449,18140,19212,1.1559,2
1019,22.09.2022 14:00:00.000 GMT-0700,19212,19465,18501,18811,0.8004,0
1020,23.09.2022 14:00:00.000 GMT-0700,18811,19335,18743,19086,0.0584,0


In [32]:
def SIGNAL():
    return df.signal

In [33]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [34]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):
    initsize = 0.05
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        if(self.signal1>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0):
            self.mysize=self.mysize*2
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            self.mysize=self.initsize
            
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 450e-4
            tp1 = self.data.Close[-1] + 450e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 450e-4
            tp1 = self.data.Close[-1] - 450e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

In [36]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=100, margin=1/50, commission=.00)
stat = bt.run()
stat



C:\Users\deerr\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\deerr\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  This is separate from the ipykernel package so we can avoid doing imports until


Start                                     0.0
End                                    1021.0
Duration                               1021.0
Exposure Time [%]                         0.0
Equity Final [$]                        100.0
Equity Peak [$]                         100.0
Return [%]                                0.0
Buy & Hold Return [%]              126.628997
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [37]:
bt.plot()

Row(id='2007', ...)